# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
# Your code here
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
len     60 non-null float64
supp    60 non-null object
dose    60 non-null float64
dtypes: float64(2), object(1)
memory usage: 1.5+ KB


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [8]:
# Your code here
formula = 'len ~ C(supp) + C(dose)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ = 2)
table

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,14.016638,4.292793e-04
C(dose),2426.434333,2.0,82.810935,1.871163e-17
Residual,820.425000,56.0,NaN,NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [ ]:
# Your comment here
# Small p-value for both, so reject null hypothesis. Dosage looks to be more impactful.

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [15]:
# Your code here
OJ = df[df['supp'] == 'OJ']['len']
OJ.describe()

count    30.000000
mean     20.663333
std       6.605561
min       8.200000
25%      15.525000
50%      22.700000
75%      25.725000
max      30.900000
Name: len, dtype: float64

In [16]:
VC = df[df['supp'] == 'VC']['len']
VC.describe()

count    30.000000
mean     16.963333
std       8.266029
min       4.200000
25%      11.200000
50%      16.500000
75%      23.100000
max      33.900000
Name: len, dtype: float64

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [20]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
import scipy.stats as stats

t_val, p_val = stats.ttest_ind(OJ,VC)
print(f'The p-value is {round(p_val,5)}')

The p-value is 0.06039


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [21]:
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ = 2)
table

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,3.668253,0.060393
Residual,3246.859333,58.0,NaN,NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [22]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [31]:
from itertools import combinations

In [50]:
groups = [group[0] for group in df.groupby(['supp', 'dose'])['len']]
combos = list(combinations(groups, 2))
for item in combos:
    supp1 = item[0][0]
    supp2 = item [1][0]
    dose1 = item [0][1]
    dose2 = item [1][1]
    Sample1 = df[(df['supp'] == supp1) &(df['dose'] == dose1) ]['len']
    Sample2 = df[(df['supp'] == supp2) & (df['dose'] == dose2) ]['len']
    t_val, p_val = stats.ttest_ind(Sample1,Sample2)
    print(f'{item} p-value is {p_val}')

(('OJ', 0.5), ('OJ', 1.0)) p-value is 8.357559281443774e-05
(('OJ', 0.5), ('OJ', 2.0)) p-value is 3.4018585295016214e-07
(('OJ', 0.5), ('VC', 0.5)) p-value is 0.005303661339923052
(('OJ', 0.5), ('VC', 1.0)) p-value is 0.04223992429368205
(('OJ', 0.5), ('VC', 2.0)) p-value is 7.025409196997986e-06
(('OJ', 1.0), ('OJ', 2.0)) p-value is 0.03736279585664383
(('OJ', 1.0), ('VC', 0.5)) p-value is 1.3372624230559434e-08
(('OJ', 1.0), ('VC', 1.0)) p-value is 0.0007807261651774468
(('OJ', 1.0), ('VC', 2.0)) p-value is 0.09583711277517494
(('OJ', 2.0), ('VC', 0.5)) p-value is 1.3381068810881244e-11
(('OJ', 2.0), ('VC', 1.0)) p-value is 2.3131084633597503e-07
(('OJ', 2.0), ('VC', 2.0)) p-value is 0.9637097790041267
(('VC', 0.5), ('VC', 1.0)) p-value is 6.492264598157612e-07
(('VC', 0.5), ('VC', 2.0)) p-value is 4.957285658438862e-09
(('VC', 1.0), ('VC', 2.0)) p-value is 3.397577925539582e-05


## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.